# Introduction: Load Modules and Import Audio

### Load Modules

In [54]:
###########################################################################################
# uncomment the following 2 lines to reload the modules automatically,
# such that changes to Plot_functions.py are reloaded without restarting the kernel!
#
%load_ext autoreload
%autoreload 2
#
###########################################################################################
    
import matplotlib.pyplot as plt
import ipywidgets as widgets
import librosa
import IPython.display as ipd
import numpy as np
import parselmouth
import soundfile as sf
import bokeh
import sounddevice as sd
import time as clock
import Plot_functions as SCplot # imports the necessary plot functions
import Helper_functions as SCtools # imports the necessary helper functions

from pathlib import Path
from scipy import signal, fft, ifft

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load and Playback Audio File

In [ ]:
# Choose an Audio File
fileName = 'f116.wav'
#fileName = 'f216.wav'
#fileName = 'a_8000.wav'
#fileName = '1000hz_3sec.wav'

snd, audio1, fs, filePath = SCtools.import_sound_file(fileName)
ipd.Audio(filePath) # show audio player

# Experiment 1: Time-Domain Analysis
## Short-Time Average Energy (Intensity)

In this experiment we will discuss the intensity curve in context with speech analysis and investigate the effects of different window lengths. To calculate the intensity curve, we firstly use our own function based on pythons 'signal' module. After that, we will move on to the methods provided by the library 'praat-parselmouth', whith provides the functionality of the software Praat in Python using the original Praat functions, which are written in C++.
> Praat documentation: <https://www.fon.hum.uva.nl/praat/manual/Intro.html>


### Own Implementation using pythons 'signal' library
To calculate the short-time average intensity, we implement the function SC_intensity(). In this function, the signal gets averaged using a Gaussian window. A Gaussian window need 2 parameters, see [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.windows.gaussian.html#scipy.signal.windows.gaussian), which are the window length in samples and the standard deviation. Unfortunately, Praat's documentation is not very helpful here, as it provides no information about the standard deviation. Therefore, we go over to [Matlab's documentation of the Gaussian window](https://de.mathworks.com/help/signal/ref/gausswin.html) where we seek help to find a value for the standard deviation, for which we trust Matlab's default value of alpha=2.5.

### Task 1.1: Find a suitable value for the standard deviation with the help of Matlab's documentation of the Gaussian window.
#### Expected Answer:
- the student should recognize from Matlabs documentation that a reasonable value for the standard deviation (or std in the code below) is given as sigma = (L-1)/(2*alpha) with alpha=2.5, where L is the length of the window in samples

In [ ]:
windowLength = 20 #millisecond

#Calculate minimum pitch from window length
minimumPitch = 1000/windowLength

def SC_intensity(sound, minimumPitch, fs):
    #winLenEffective = np.round(3.2/minimumPitch * fs)  # Window length in samples; from Praat documentation; for pitch-synchronous intensity ripple
    winLenEffective = np.round(1/minimumPitch * fs)
    
    # TODO: find a suitable value for the standard deviation std
    # std = ?
    alpha = 2.5  # default value from Matlab documentation
    std = (winLenEffective-1)/(2*alpha)  # standard deviation from Matlab documentation
    # TODO_END
    win = signal.windows.gaussian(winLenEffective, std) # Gaussian window
    #win = signal.windows.kaiser(winLenEffective, 20/np.pi) # Kaiser window - alternative to Gaussian window
    
    sound = np.square(sound-np.mean(sound)) 

    intensity = np.convolve(sound, win, mode='valid') /np.sum(win)
    
    intensity = 10*np.log10(intensity/(4e-10)) # conversion to dB ; norm to (20 muPa)^2

    print("SC_intensity: Intensity has {} samples".format(intensity.size))
    winLenEffectiveTime = winLenEffective / fs
    return intensity, win, winLenEffectiveTime

SC_intensity, gaussWin, winLenEffectiveTime = SC_intensity(np.squeeze(snd.values), minimumPitch, fs)

# Plot window function
plottitle = "Gaussian Window for Averaging with SC_intensity()"
dt_win = np.arange(0, gaussWin.size) / fs  # time axis for Gaussian Window
p_window = SCplot.get_plot_window(gaussWin, dt_win, plottitle)

# Plot intensity curve
snd_values = np.squeeze(snd.values)
dt_snd = np.arange(0, snd_values.size) / fs
dt_SC_intensity = np.arange(0,SC_intensity.size) / fs 
dt_SC_intensity = dt_SC_intensity + winLenEffectiveTime/2 # shift to center the intensity curve bins in the windows

plottitle = 'Intensity calculated with SC_intensity() - File: ' + fileName
p_intensity = SCplot.get_plot_intensity(snd_values, dt_snd, SC_intensity, dt_SC_intensity, plottitle)

SCplot.plot_in_subplots(p_window, p_intensity)

### Praat-Parselmouth implementation
Also, the library praat-parselmouth provides us with functionality to calculate the intensity. To calculate the intensity with parselmouth, the member function to_intensity() is used, which takes the minimum pitch as an input argument. To compare parselmouth's to_intensity() with our custom SC_intensity(), we use the same input parameters as before.

In [ ]:
# PM_intensity = snd.to_intensity(minimum_pitch=minimumPitch, subtract_mean=False)  # intensity calculation with parselmouth's function
PM_intensity = snd.to_intensity(minimum_pitch=minimumPitch, subtract_mean=False, time_step=1/fs)  # intensity calculation with parselmouth's function
print("PM_intensity: Intensity has {} samples".format(PM_intensity.get_number_of_frames()))

dt_PM_intensity = PM_intensity.x_grid()[:-1]
PM_intensity_val = np.squeeze(PM_intensity.values)
dt_snd = snd.x_grid()[:-1]

plottitle = "Intensity calculated with parselmouth's to_intensity() - File: " + fileName
SCplot.get_plot_intensity(np.squeeze(snd.values), dt_snd, PM_intensity_val, dt_PM_intensity, plottitle, showPlot=True)

### Comparing the two intensity curves
Compare the two intensity curves in the following plot:

In [ ]:
# plot the 2 intensity curves in one plot
plottitle = "Comparison of both intensity curves - File: " + fileName
SCplot.plot_two_intensity_curves(dt_PM_intensity, PM_intensity_val, dt_SC_intensity, SC_intensity, plottitle)

### Task 1.2: Describe the key differences of the intensity curve calculated with your own function SC_intensity() to parselmouth's to_intensity().
#### Expected Answers:
- different value range
- parselmouths intensity is more sparsely sampled
- parselmouths intensity has not the same time range as the audio file, whereas the custom intensity covers the whole time range of the audio file

### Task 1.3: What happens to the intensity curve, if we use different window sizes?
#### Expected answers:
- short windows allow more fluctuations of the intensity curve
- long windows lead to more smoothing of the intensity curve, thus the fluctuations of the intensity curve will become smaller


### Task 1.4: Is there a way to alter the intensity curve calculated with your own function SC_intensity(), such that it matches the intensity curve calculated with parselmouth better?
> Hint: see <https://www.fon.hum.uva.nl/praat/manual/Sound__To_Intensity___.html>
#### Expected Answer:
- the student should recognize that parselmouth uses an effective window length that is 3.2 times the original window length
- the student should modify the function SC_intensity() such that it uses this effective window length

# Experiment 2: Frequency-Domain Analysis
In this experiment, we analyze the sound file in frequency domain, for example using spectrograms. Again, we implement a spectrogram using Python's 'signal' model, and compare it with Praat-Parselmouth's spectrogram.
### Load and Playback Audio File

In [ ]:
# Choose an Audio File
#fileName = 'f116.wav'
#fileName = 'f216.wav'
fileName = 'a_8000.wav'
#fileName = '1000hz_3sec.wav'

snd, audio1, fs, filePath = SCtools.import_sound_file(fileName)
ipd.Audio(filePath) # show audio player

### Own implementation using Python's 'signal' module
First, we start by calculating a spectrogram using the method scipy.signal.spectrogram().

In [ ]:
windowlengthSec = 30 #ms
windowlength = np.round(fs * windowlengthSec/1000).astype(int)
#windowlength = 2048
print('Window Length in samples:', windowlength)
#overlap = windowlength-1
overlap = np.round(windowlength / 2)

# Gaussian window
std = (windowlength - 1)/(2*2.5)  # see Task 1.1
window = ('gaussian', std) # tuple for signal.spectrogram

#window = 'hann' # Hann window - alternative to Gaussian window

SC_fVec, SC_tVec, SC_spectroData = signal.spectrogram(audio1, fs=fs, window=window, noverlap=overlap, nperseg=windowlength, return_onesided=True, scaling='spectrum', mode='magnitude')

SC_spectroDataDB = 20*np.log10(SC_spectroData / np.max(SC_spectroData))

# plot interactive spectrogram

# TODO: choose slider Default-Value which produces a reasonable plot
defaultValue = 0
# TODO_END

plottitle = "Custom Spectrogram of Sound Sample - File: " + fileName
SC_timeWidget = SCplot.plot_interactive_spectrogram(SC_spectroDataDB, SC_tVec, SC_fVec, plottitle,defaultValue)
widgets.HBox([SC_timeWidget])

### Praat-Parselmouth implementation
Now we use parselmouths to_spectrogram() to calculate a spectrogram. For Plotting the spectrogram, we use our custum function plot_interactive_spectrogram().

In [ ]:
windowlengthSec = 70 #ms
maximumFrequency = 5000

def PM_get_spectrogram(snd, windowLengthMS=30, maximumFrequency=5000):
    spectrogram = snd.to_spectrogram(window_length=windowLengthMS/1000, maximum_frequency=maximumFrequency)
    PM_spectroData = spectrogram.values
    PM_tVec = spectrogram.ts()
    PM_fVec = spectrogram.ys()
    PM_spectroDataDB = 10*np.log10(PM_spectroData / np.max(PM_spectroData))
    return PM_spectroDataDB, PM_tVec, PM_fVec

PM_spectroDataDB, PM_tVec, PM_fVec = PM_get_spectrogram(snd, windowLengthMS=windowlengthSec, maximumFrequency=maximumFrequency)

plottitle = "Parselmouth Spectrogram of Sound Sample - File: " + fileName
PM_timeWidget = SCplot.plot_interactive_spectrogram(PM_spectroDataDB, PM_tVec, PM_fVec, plottitle, defaultValue)
widgets.HBox([PM_timeWidget])

### Task 2.1: Effects of different window lengths
Modify the window length in the spectrogram generated using Praat-Parselmouth and set it to the following values:
- 10 ms
- 30 ms
- 70 ms

Describe the effects you see in the spectrogram and in the spectrum. What could 'wide band analysis' and 'narrow band analysis' mean in this context?
#### Expected answers:
- the shorter the window, the better is the time resolution, but the frequencfy resolution gets worse
- if we use long windows, the frequency resolution gets better, but the time resolution gets worse
- 'wide band analysis' refers to spectrogram analysis using short windows, resulting in a good time resolution, but a poor frequency resolution (i.e. the frequencies are only displayed as 'wide bands')
- 'narrow band analysis' refers to spectrogram analysis using long windows, thus with a good frequency resolution, but a poor time resolution

### Task 2.2: Interpretation of the spectrum and the spectrogram
What information can be retrieved from the spectrogram? Think of parameters of the signal that are easily represented in frequancy domain.
#### Expected answers:
- fundamental frequency f0
- harmonic structure of the signal, formant structure
- what kind of voice signal is it? (vowel, consonant or fricative)

## Analysis of f0 and formants
To analyze the Formants, we use praat-parselmouths formant analysis methods. We analyse the fundamental frequency f0 and additionally the first four formants F1,...,F4.

Select sound file to analyze:

In [ ]:
# Choose an Audio File
#fileName = 'f116.wav'
#fileName = 'f216.wav'
fileName = 'e_11025.wav'
#fileName = 'a_8000.wav'
#fileName = '1000hz_3sec.wav'

snd, _, fs, filePath = SCtools.import_sound_file(fileName)
ipd.Audio(filePath) # show audio player

To analyze the fundamental frequency and the formants, we use the Praat-parselmouth methods to_pitch() for the fundamental frequency and to_formant_burg() for the formants. For the formant analysis, the BURG-algorithm is used (more information about this algorithm, which is based on LPC internally, is [here](https://asa.scitation.org/doi/10.1121/1.2003944)).

In [ ]:
def PM_get_f0_and_formants(snd, pitchLo = 75, pitchHi = 400, pitchTimeStep = 30, FormantWindowLength = 30,
                           maxNumberFormants = 4, maxFormantFreq = 4000):

    # evaluate pitch using praat-parselmouth
    PM_pitch = snd.to_pitch(pitch_floor = pitchLo, pitch_ceiling=pitchHi)

    pitch_tVec = PM_pitch.ts()
    pitchValues = np.zeros_like(pitch_tVec)

    for timeIdx, time in enumerate(pitch_tVec):
        pitchValues[timeIdx] = PM_pitch.get_value_at_time(time=time)

    # evaluate formants using praat-parselmouth (here: BURG-algorithm is used)
    PM_formants = snd.to_formant_burg(maximum_formant=maxFormantFreq, window_length=FormantWindowLength/1000,
                                      max_number_of_formants=maxNumberFormants)

    formant_tVec = PM_formants.ts()
    formantValues = np.zeros((maxNumberFormants,formant_tVec.size))

    for timeIdx, time in enumerate(formant_tVec):
        for formantIdx in range(maxNumberFormants):
            formantValues[formantIdx,timeIdx] = PM_formants.get_value_at_time(formant_number=formantIdx+1, time=time)
    
    return pitchValues, pitch_tVec, formantValues, formant_tVec

# calculate spectrogram using praat-parselmouth
spectroDataDB, spec_tVec, spec_fVec = PM_get_spectrogram(snd, 30, maximumFrequency)

# calculate f0 and formants using praat-parselmouth
pitchValues, pitch_tVec, formantValues, formant_tVec = PM_get_f0_and_formants(snd)

# plot spectrogram with f0 and formants
plottitle = "Spectrogram, f0 and Formants of Sound Sample - File: " + fileName
SCplot.plot_spectrogram_with_formants(spectroDataDB, spec_tVec, spec_fVec, formantValues, formant_tVec,plottitle,
                                      pitchValues=pitchValues, pitch_tVec=pitch_tVec)

### Formants F1 & F2 in the Vowel chart
Especially with vowels, the first two formants F1 and F2 are very important in order to distinguish one vowel from another. Therefore, F1 and F2 can be displayed in the so-called vowel chart. Below is an interactive vowel chart where you can select a region in the audio file from which the detected formants are displayed in the vowel chart.
The vowel chart used here is by Sendlmeier et. al - for more information see [here](https://www.kw.tu-berlin.de/fileadmin/a01311100/Formantkarten_des_deutschen_Vokalsystems_01.pdf).

In [ ]:
dt_snd = snd.x_grid()[:-1]
F1 = formantValues[0,:]
F2 = formantValues[1,:]


plottitle = "Detected Formants F1 & F2 in the Vowel Chart - File: " + fileName
PM_timeWidget = SCplot.plot_F1_F2_in_vowel_chart(np.squeeze(snd.values), dt_snd, F1, F2, formant_tVec, plottitle)
widgets.HBox([PM_timeWidget])

# Experiment 3: Estimation of Vocal Tract using Cepstrum and LPC
## LPC: Levinson Durbin Algorithm
The Levinson.Durbin-Algorithm is an elegant procedure to calculate the LPC-coefficients recursively. This implementation uses the whole signal for processing.

### Task 3.1: Analysing with LPC:
- If the whole signal is used for LPC analysis, which aspects has the soundfile to fulfill? 
- How can sentences be analysed?

#### Expected Answers:
- Estimating the Vocal Tract Filter of speech signals (more generally: seperating the source signal from the transfer path) requires a stationary signal. If the transfer path changes inside the signal to analyse, the results are not representative. 
- Therefore, sentences have to be buffered in order to be analysed!

Choose a suitable sound file for the Levinson-Durbin-Algorithm from the code lines below, or choose another from the file directory. You can listen to a file by uncommenting and run the related line in the code below. 

##### Keep in mind that the whole signal is used for this particular implementation of the LPC. 

In [41]:
# choose an audio file

#fileName = 'f116.wav'
#fileName = 'f216.wav'
fileName = 'a_8000.wav'
#fileName = '1000hz_3sec.wav'

_, sig, fs, filePath = SCtools.import_sound_file(fileName)
ipd.Audio(filePath) # show audio player

a_8000.wav loaded with sampling frequency f_s = 8000


Now set the LPC order below ("lpcOrder"), and run the code. The auto-correlation, vocal tract filter spectrum and its Z-plane should be displayed. If there are confusingly many coefficients plotted, choose a suitalbe number of coefficients to display ("coefficientBoundary"). Run the code again and take a look at the results!

Take a look at the funktion "autocorr()" and check how the auto-correlation is calculated. For the dokumentation of "np.correlate()" click [here](https://numpy.org/doc/stable/reference/generated/numpy.correlate.html).

The grey line displayed in back of the "Vocal Tract Filter Spectrum" - plot ist the power spectral density. 

### Task 3.2: Pre-Emphasis Filter
A pre-emphasis filter can significantly improve the result of a LPC analysis. Use the predefined function "pre_emphasis_filtering(sig, fs)" to implement the pre-emphasis filtering before calculating the LPC coefficients.

- Implement the pre-emphasis filter and compare the results.
- What does a pre-emphasis filter do, and why does the algorithm perform better/worse?

#### Expected Answers:
- Comparing the results, there is a significant drop to high frequencies in the vocal tract spectrum of the estimation without pre-emphasis. Also the autocorrelation of the pre-emphasis filtered signal drops to much smaller values (with pre-emph.). The results of the estimation with pre-emphasis seem more reliable.
- The pre-emphasis filter whitens the spectrum and therefore decorrelates the signal (for time-lags bigger than 0). The Levinson-Durbin Algorithm is constructed to perform best with decorrelated signals. In case of speech, the main signal energy is located in lower frequencies. Therefore the pre-emphasis filter is a low-cut filter, in this case a first order low-cut filter. 


In the resulting plot below, the iterations of the Levinson-Durbin algorithm can be stepped through. The approach of the coefficients can be observed.

### Task 3.3: High Order LPC
Choose a LPC order so high, that you can see peaks unusual to a spectral envelope.

- How can you explain the prominent peaks in the estimated vocal tract filter spectrum for high LPC orders?

#### Expected Answer:
- These peaks represent the harmonics of the speech signal. For too large numbers of coefficients the estimation includes harmonics.

In [59]:
def autocorr(x, norm=True):
# Calculate Autocorrelation:
# input:
#   x...input signal
#   norm...(bool) normalize autocorrelation (default True)
#
# return:
#   result...autocorrelation of x (only first half)
#
    result = np.correlate(x, x, mode='full')
    result = result[result.size // 2:]
    if norm:
        result = result/result[0]
    return result

def pre_emphasis_filtering(sig, fs, inv=False):
# Pre-Emphasis Filtering:
# input:
#   sig...signal to filter
#   fs...sampling frequency
#   inv...(bool) inversely filtering (default False)
#
# return:
#   sigPreEmphasis...filtered signal
#
    fPreEmph = 10
    # alpha is calculated as in egifa!
    alpha = np.exp(-2*np.pi*fPreEmph/fs)
    if inv:
        # 1. order Low-Cut
        sigPreEmphasis = signal.lfilter([1],np.append([1], -alpha),sig)
    else:
        sigPreEmphasis = signal.lfilter(np.append([1], -alpha),[1],sig)
        
    return sigPreEmphasis

###########################################################################################################################

# TODO: set the LPC order and if necessary limit the number of coefficients:
lpcOrder = 100
coefficientBoundary = lpcOrder
# TODO_END

E = np.zeros(lpcOrder-1) # define size of error vector
K = np.zeros(lpcOrder-1) # define size of reflection coefficient vector
a_all = np.zeros([lpcOrder-1, lpcOrder]) # define size of filter coefficient matrix (coefficient vector each row)

# TODO: implement pre-emphasis filtering
sigPreEmphasis = pre_emphasis_filtering(sig, fs) # pre-emphasis filtering to whiten the speech signal before analyzing it

R = autocorr(sigPreEmphasis) # calc. the autocorrelation of the signal (only significant part)
#R = autocorr(sig) # calc. the autocorrelation of the signal (only significant part)
# TODO_END

SCplot.plot_autocorr(R, 'Auto-Correlation') # plot auto-correlation

K[0] = R[1]/R[0] # initialize reflection coefficient
a = K[0] # initialize lpc coefficient
a_all[0,0] = 1
a_all[0,1] = -a # save initial lpc coefficient as IIR filter coefficient
E[0] = R[0] # initialize error

for i in range(1,lpcOrder-1):
    
    K[i] = (R[i+1] - np.dot(a, R[1:i+1])) / E[i-1] # calc new reflection coefficient
    
    a = np.append(K[i], a-K[i]*np.flip(a)) # calc new lpc coefficients
    
    E[i] = E[i-1] * (1-(abs(K[i])**2)) # calc new error
    
    a_all[i,0:i+2] = np.append([1], -np.flip(a)) # save current lpc coefficients as IIR filter coefficients

    
SC_iterationWidget = SCplot.plot_interactive_filter_zplane(a_all, coefficientBoundary, fs, get_spectrum(np.append(R, np.flip(R[0:-1])), 2*len(R)-1)+40, 'Vocal Tract Filter Spectrum')
widgets.HBox([SC_iterationWidget])

In the section below the time signals and spectra of the original signal and the signal inversly-filtered by the estimated vocal-tract (so the estimated glottis signal!) are displayed.

With the interactive plot above, select a reasonable LPC order (number of iterations). By writing the parameters into the section below ("selectedIteration", "limitCoefficients"), the figures correspond to the selected filter. Typically the number of coefficients coincides with the LPC order. If so, you may not want limit the number of coefficients (LPC order = number of coefficients => "limitCoefficients = 0"). 

### Task 3.4: Fundamental Frequency f0
Zoom into the Glottis time signal and measure the period using the crosshairs. Then calculate the frequency of the Glottis signal and compare it to the fundamental frequency derived from the vocal-tract estimation. If so, why do differences occur?

#### Expected Answer:
- measured from Glottis signal: T = 0.008s    ->    f0 = 125Hz
- measured from Vocal Tract: f0 = 140Hz
- The measurement from the glottis signal is a local observation in contrast to the global observation of the estimated vocal tract.

### Task 3.5: Think of applications for LPC!
#### Expected Answer:
- vocoder
- telephone (speech coding)
- speech analysis
- speech synthesis

In [6]:
# Calclate Spectrum:
# input:
#   signalData...input signal
#   NFFT...fft order
#   window...(bool) window signal with hann window (default True)
#   logarithm...(bool) returns the spectrum in dB if true (default True)
#
# return:
#   spetrum...positive frequencies of spectrum
#
def get_spectrum(signalData, NFFT, window = True, logarithm = True):
    dataVec = np.zeros(NFFT)
    if len(signalData) > NFFT:
        dataVec = signalData[0:NFFT-1]
        win = signal.hann(NFFT)
    else:
        dataVec[0:len(signalData)] = signalData
        win = signal.hann(len(signalData))
        win = np.append(win, np.zeros(NFFT-len(signalData)))
    if window:
        dataVec = dataVec*win
    if logarithm:
        spectrum = 20*np.log10(abs(fft(dataVec)))
    else:
        spectrum = abs(fft(dataVec))
    return spectrum[0:round((len(spectrum)+1)/2)]

###########################################################################################################################

# TODO: select reasonalbe filter coefficients from the interactive plot
selectedIteration = 30
# TODO_END

# TODO: limit number of filter coefficients (if 0, no limit an number of coefficients equals number of iterations)
limitCoefficients = 0
# TODO_END

selectedFilter = np.trim_zeros(a_all[selectedIteration], 'b')
if limitCoefficients < len(selectedFilter) and limitCoefficients > 0:
    selectedFilter = selectedFilter[0:limitCoefficients]

#print(selectedFilter)

filteredSignal = signal.lfilter([1], selectedFilter, sig)
filteredSignal = filteredSignal / max(abs(filteredSignal)) # normalize
inverseFilteredSignal = signal.lfilter(selectedFilter, [1], sig)
inverseFilteredSignal = inverseFilteredSignal / max(abs(inverseFilteredSignal)) # normalize
inverseFilteredSignalPreEmph = pre_emphasis_filtering(inverseFilteredSignal, fs) # preEmph filtered to lose the speech characteristec level drop to high frequencies
inverseFilteredSignalPreEmph = inverseFilteredSignalPreEmph / max(abs(inverseFilteredSignalPreEmph)) # normalize

SCplot.plot_time_signal(sig, fs, 'Original Time Signal')
#SCplot.plot_time_signal(filteredSignal, fs, 'Vocal Tract Time Signal')
SCplot.plot_time_signal(inverseFilteredSignal, fs, 'Glottis Time Signal (not pre emphasis filtered)')

sigSpectrum = get_spectrum(sig, len(sig), window=True)
filteredSignalSpectrum = get_spectrum(filteredSignal, len(filteredSignal), window=True)
inverseFilteredSignalSpectrum = get_spectrum(inverseFilteredSignal, len(inverseFilteredSignal), window=True)

freqVector = np.linspace(0, fs/2, round(len(sig)/2+1))
SCplot.plot_spectrum(get_spectrum(sig, len(sig), window=True), freqVector, 'Original Signal - Spectrum')
SCplot.plot_spectrum(get_spectrum(sigPreEmphasis, len(sig), window=True), freqVector, 'Pre Emphasis Filtered Signal - Spectrum')
SCplot.plot_spectrum(get_spectrum(inverseFilteredSignalPreEmph, len(inverseFilteredSignalPreEmph), window=True), freqVector, 'Glottis - Spectrum (pre emphasis filtered)')


## Cepstrum Analysis
In this part the source, filter seperation is carried out by a cepstral analysis.
Therefore the cepstrum of the signal is calculated (http://piotr.majdak.com/alg/VO/sourcefilter.pdf, page 18).
In the cepstrum the slowly changing parts of the spectrum are located in the lower quefrencies and the fast oscillating parts in higher quefrencies. Therefore we can calculate the spectral envelope by liftering the cepstrum with a rectangular lifter. Also we can directli derive the spectral fine structure by liftering. 

### Task 3.7: Source-Filter-Seperation through Liftering
- Choose the lifter length ("lifterLength") in samples so a reasonalbe vocal tract filter results. To do so, test diffrent lifter lenghts and monitor the plots given below.
- Compare both vocal tract estimations and discuss the differences.

#### Expected Answer:
- A lifter length arround 30 gives a reasonable vocal tract filter estimation. Too large lifter lenghts result in spectra with speech harmonics visible and so a glottis signal spectrum consisting of noise only. Too small lifter lengths give a glottis signal spectrum with some envelope stil visible.

In [19]:
# Calculate Cepstrum:
# input:
#   spectrum...spectrum input (no logarithm)
#   logBase10...(bool) if true log10 is used, if false the natural log is used (default True)
#   mirrorSpectrum...(bool) mirrors spectrum if treu, use for positive frequency input (default False)
#
# return:
#   cepstrum...cepstrum of positive and negative quefrencies
#
def get_cepstrum(spectrum, logBase10 = True, mirrorSpectrum = False):
    if mirrorSpectrum:
        spectrum = np.append(spectrum, np.flip(spectrum[0:-1]))
    if logBase10:
        spectLog = np.log10(abs(spectrum))
    else:
        spectLog = np.log(abs(spectrum))
    #cepstrum = 4* ifft(spectLog)**2
    cepstrum = ifft(spectLog)
    return cepstrum
        
###########################################################################################################################

# TODO: set length of cepstrum lifter in samples
lifterLength = 30
# TODO_END

#sig = sig - np.mean(sig)

sigLength = len(sig)
fftLength = sigLength
timeVector = np.linspace(0, sigLength/fs, sigLength)
freqVector = np.linspace(0, fs/2, round(fftLength/2+1))


spectrum = get_spectrum(sig, len(sig), window = True, logarithm=False)

cepstrum = get_cepstrum(spectrum, mirrorSpectrum=True, logBase10=True)
cepstrumForEnvelope = get_cepstrum(spectrum, mirrorSpectrum=True, logBase10=True)
cepstrumForFineStructure = get_cepstrum(spectrum, mirrorSpectrum=True, logBase10=True)
SCplot.plot_cepstrum(20*np.log10(abs(cepstrum[0:round(len(cepstrum)/2+1)])), range(len(cepstrum[0:round(len(cepstrum)/2+1)])), lifterLength, 'Signal Cepstrum', lifterLP=True)

cepstrumForEnvelope[lifterLength:-lifterLength] = 0 #liftering

cepstrumForFineStructure[0:lifterLength] = 0 #liftering
cepstrumForFineStructure[-lifterLength:] = 0 #liftering

inverseCepstrumForEnvelope = 10**(fft(cepstrumForEnvelope))
inverseCepstrumForEnvelope = inverseCepstrumForEnvelope[0:round(len(inverseCepstrumForEnvelope)/2)]

SCplot.plot_spectrum(20*np.log10(abs(inverseCepstrumForEnvelope)), freqVector, 'Spectrum Envelope based on Cepstrum')

inverseCepstrumForFineStructure = 10**(fft(cepstrumForFineStructure))
inverseCepstrumForFineStructure = inverseCepstrumForFineStructure[0:round(len(inverseCepstrumForFineStructure)/2)]

SCplot.plot_spectrum(20*np.log10(abs(inverseCepstrumForFineStructure)), freqVector, 'Spectrum Fine Structure based on Cepstrum')


# Experiment 4: Formant structure Analysis

In this exercise you need to record your own speech-samples. Before recording is possible the input/output sounddevices have to be set. Use the first cell to display all possible sound-devices and select the wanted devices by assigning the sounddevice-indices to the variables 'input_device' and 'output_device'.

Then you can use the given record, play to record your own speech-samples.

For this experiment please record 3 different versions of one sentence and analyze the time-domain signal and its formant and f0 structure (as already done in Experiment 2) and plot the results in separate cells. By plotting the results in separate cells it is possible to always use the same record button and audio-array for different recordings.

In [ ]:
fs = 44100 #Hz

#show all possible sound-devices
sd.query_devices()

In [ ]:
#set default fs and number of channels
sd.default.samplerate = fs
num_channels = 2

# TODO: select sound-device by choosing ID of the list shown above.
input_device = 0
output_device = 1
# TODO_END

sd.default.device = [input_device,output_device]


In [ ]:
# record and playback audio
toggleRec,togglePlay,out,box_layout,indata = SCtools.get_rec_and_play_button(fs, num_channels, max_duration = 30)

widgets.HBox([toggleRec,togglePlay,out],layout=box_layout)

### Task 4.1: Recording of Standard Sentence:
Record a standard sentence and plot the time-domain signal and the spectrogram/formant/f0 plot as you have already done in Experiment 2. For the time-domain signal use the provided function 'get_plot_time_domain_sig()'. The function-arguments are described in the corresponding function's header available in the file 'plot_functions.py'. To analyze the spectrogram/frequency and f0 structure of the recorded sentence you can use the Parselmouth-analysis and the corresponding plot-functions of Experiment 2 ('PM_get_spetrogram()' and 'PM_get_f0_and_formants()'). The recorded Data is stored into the array 'recData'. Please plot the results in a separate Cell and answer the following questions:

- Are there visible formant contours?
- Can you distinguish between vowels, consonants and fricatives?

#### Expected Answers:
- Yes there should be visible formants assuming a standard sentence containing vowels was recorded.
- Yes vowels are distinguishable with a clear formant structure and fricatives don't show a clear formant-structure. For consonants like 'l', 'm' or 'n' there can be visible formant structures because they can also be used in a voiced way.

In [ ]:
#prepare recordings for plots
indata_no_Nan = SCtools.deleteNan(indata)
# the recorded signal is converted to a mono signal and stored in recData
recData = (indata_no_Nan[:,0]+indata_no_Nan[:,1])/2
rec_time = np.linspace(0,recData.shape[0]/fs,recData.shape[0])
snd = parselmouth.Sound(recData)




# TODO: plot time-domain signal   
SCplot.get_plot_time_domain_sig(recData,rec_time,'Recorded Time-Domain-Signal',showPlot=True)
# TODO_END

# TODO: calculate spectrogram using praat-parselmouth
speech_spectro, speech_spectro_t, speech_spectr_f = PM_get_spectrogram(snd, windowLengthMS=30, maximumFrequency=5000)
# TODO_END

# TODO: calculate f0 and formants using praat-parselmouth
pitchValues, pitch_tVec, formantValues, formant_tVec = PM_get_f0_and_formants(snd)
# TODO_END

# TODO: plot the spectrogram 
plottitle = 'Spectrogram, f0 and Formants of recorded Sample'
SCplot.plot_spectrogram_with_formants(speech_spectro, speech_spectro_t, speech_spectr_f, formantValues, formant_tVec,plottitle,
                                      pitchValues=pitchValues, pitch_tVec=pitch_tVec)
# TODO_END

### Task 4.2: Recording of Dada-Sentence:
Now, please go back to the "record" button and record a so called Dada-sentence but try to keep the same pitch/pitch course as in the sentence before. The Dada-sentence should not consist of words but rather of sounds like 'da', 'la', 'fa' etc. After recording please plot the time-domain signal and spectrogram/formant/f0-plot in a separate cell.
- Is the course of f0 the same as in the Recording before?
- How do the formant-structures differ?
- Are there any differences visible in the time-domain?

#### Expected Answers:
- Should be the same if it has been possible to keep the same pitch as the sentence before, if not maybe try again.
- If only one sound e.g. 'da' was used the same formant-structure should repeat itself and it should be visible how the formants of the 'a' contained in 'da', emerge from lower frequencies, due to the fact that a consonant is used before the 'a'.
- If a voiced sound like 'da' was used to record the sentence the time-domain signal should consist only of voiced-segments and no "noisy" segments. "Voiced" segments contain more energy than "noisy" whereas noisy segments of sounds like 's' or the german 'sch' fluctuate more randomly and the typical noise structure is visible in the time-domain signal. 

In [ ]:
#prepare recordings for plots
indata_no_Nan = SCtools.deleteNan(indata)
# the recorded signal is converted to a mono signal and stored in recData
recData = (indata_no_Nan[:,0]+indata_no_Nan[:,1])/2
rec_time = np.linspace(0,recData.shape[0]/fs,recData.shape[0])
snd = parselmouth.Sound(recData)

# TODO: plot time-domain signal   
SCplot.get_plot_time_domain_sig(recData,rec_time,'Recorded Time-Domain-Signal',showPlot=True)
# TODO_END

# TODO: calculate spectrogram using praat-parselmouth
speech_spectro, speech_spectro_t, speech_spectr_f = PM_get_spectrogram(snd, windowLengthMS=30, maximumFrequency=5000)
# TODO_END

# TODO: calculate f0 and formants using praat-parselmouth
pitchValues, pitch_tVec, formantValues, formant_tVec = PM_get_f0_and_formants(snd)
# TODO_END

# TODO: plot the spectrogram 
plottitle = 'Spectrogram, f0 and Formants of recorded Sample'
SCplot.plot_spectrogram_with_formants(speech_spectro, speech_spectro_t, speech_spectr_f, formantValues, formant_tVec,plottitle,
                                      pitchValues=pitchValues, pitch_tVec=pitch_tVec)
# TODO_END

### Task 4.3: Recording of whispered Sentence:
Record the previous sentence but use a whispery voice and once more please plot the time-domain signal and the spectrogram/formant/f0-plot in a separate cell.
- How does the formant-structure differ in comparison to the first Recording?
- How and why does the f0-course differ from the previouse recordings?

#### Expected Answers:
- Formant-Tracker results might vary more due to less energy in the speech signal. But some sort of Formant-structure should still be visible. 
- F0-Tracking does not work. Whispery voice does not contain a f0 structure due to the fact that the glottis does not close when speaking with a whispery voice. The glottis closing impulses are no longer present aand the Excitation signal is now noise-like. The missing glottis closing impulses lead to a missing f0 perception.

In [ ]:
#prepare recordings for plots
indata_no_Nan = SCtools.deleteNan(indata)
# the recorded signal is converted to a mono signal and stored in recData
recData = (indata_no_Nan[:,0]+indata_no_Nan[:,1])/2
rec_time = np.linspace(0,recData.shape[0]/fs,recData.shape[0])
snd = parselmouth.Sound(recData)

# TODO: plot time-domain signal   
SCplot.get_plot_time_domain_sig(recData,rec_time,'Recorded Time-Domain-Signal',showPlot=True)
# TODO_END

# TODO: calculate spectrogram using praat-parselmouth
speech_spectro, speech_spectro_t, speech_spectr_f = PM_get_spectrogram(snd, windowLengthMS=30, maximumFrequency=5000)
# TODO_END

# TODO: calculate f0 and formants using praat-parselmouth
pitchValues, pitch_tVec, formantValues, formant_tVec = PM_get_f0_and_formants(snd)
# TODO_END

# TODO: plot the spectrogram 
plottitle = 'Spectrogram, f0 and Formants of recorded Sample'
SCplot.plot_spectrogram_with_formants(speech_spectro, speech_spectro_t, speech_spectr_f, formantValues, formant_tVec,plottitle,
                                      pitchValues=pitchValues, pitch_tVec=pitch_tVec)
# TODO_END

# (Bonus) Experiment 5: Record your own formant sample
Use the provided 'Record' and 'Play' Button to record your own formant samples and visualize the results with the help of the previously used plot function 'plot_F1_F2_in_vowel_chart()' (see Experiment 2).

###### Hint: 
Make shure to limit the recording towards the voiced parts to avoid wrong formant detections. To do so the time-widget included in 'plot_F1_F2_in_vowel_chart()' can be a very helpful tool. If the slider does not respond whilst dragging it try to run the cell again! 

In [ ]:
# record and playback audio
fs = 11025 # lower sampling rate in order to reduce computational effort. 
sd.default.samplerate = fs
toggleRec,togglePlay,out,box_layout,indata = SCtools.get_rec_and_play_button(fs, num_channels, max_duration = 30)

widgets.HBox([toggleRec,togglePlay,out],layout=box_layout)

### Task 5.1: Recording of single vowel:
Try to record simple formant samples which contains a single vowel ('a','e','i','o'...).
- Do the results match the given F1/F2 chart? Please plot a recorded vowel for which the analysis matches the given chart.

#### Expected Answer:
- Vowels like 'i' or 'e' have proven themselves to produce good results which match the given chart reasonably well.

In [ ]:
#prepare recordings for plots
indata_no_Nan = SCtools.deleteNan(indata)
#convert to mono
recData = (indata_no_Nan[:,0]+indata_no_Nan[:,1])/2
rec_time = np.linspace(0,recData.shape[0]/fs,recData.shape[0])
snd = parselmouth.Sound(recData,sampling_frequency=fs)
# calculate f0 and formants using praat-parselmouth
pitchValues, pitch_tVec, formantValues, formant_tVec = PM_get_f0_and_formants(snd)

#  TODO: extract the array containing the time axis and the formants F1 and F2 
dt_snd = snd.x_grid()[:-1]
F1 = formantValues[0,:]
F2 = formantValues[1,:]
# TODO_END

# TODO: Plot the recorded Formant inside the F1/F2 Vowel Chart (see Experiment 2)
plottitle = "Recorded and detected Formants F1 & F2 in the Vowel Chart"
PM_timeWidget = SCplot.plot_F1_F2_in_vowel_chart(np.squeeze(snd.values), dt_snd, F1, F2, formant_tVec, plottitle)
widgets.HBox([PM_timeWidget])
# TODO_END

### Task 5.2: Recording of a diphtong:
Try to record a formant samples which contains a diphtong vowel ('ai','au','ei','ou'...).
- Again please visualize the tracked formants with the given plot-function in the F1/F2 vowel chart (see Experiment 2). Is the course of the changing formants visible within the vowel-chart?

#### Expected Answer:
- Works very well if time range is set to limit sample towards voiced regions.

In [ ]:
#prepare recordings for plots
indata_no_Nan = SCtools.deleteNan(indata)
#convert to mono
recData = (indata_no_Nan[:,0]+indata_no_Nan[:,1])/2
rec_time = np.linspace(0,recData.shape[0]/fs,recData.shape[0])
snd = parselmouth.Sound(recData,sampling_frequency=fs)
# calculate f0 and formants using praat-parselmouth
pitchValues, pitch_tVec, formantValues, formant_tVec = PM_get_f0_and_formants(snd)

#  TODO: extract the array containing the time axis and the formants F1 and F2 
dt_snd = snd.x_grid()[:-1]
F1 = formantValues[0,:]
F2 = formantValues[1,:]

# TODO: Plot the recorded Formant inside the F1/F2 Vowel Chart (see Experiment 2)
plottitle = "Recorded and detected Formants F1 & F2 in the Vowel Chart"
PM_timeWidget = SCplot.plot_F1_F2_in_vowel_chart(np.squeeze(snd.values), dt_snd, F1, F2, formant_tVec, plottitle)
widgets.HBox([PM_timeWidget])
# TODO_END